In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import pandas as pd
from library.classifiers import *

In [2]:
df = sns.load_dataset("iris")

dic = {'setosa': 0, 'versicolor': 1, 'virginica': 2}
df['species'] = df['species'].map(dic)

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
def split(df, class_feature):
    n_classes = df[class_feature].unique().size
    train_list = []
    test_list = []

    for i in range(n_classes):
        df_sliced = df[df['species'] == i]
        n_rows = df_sliced.shape[0]
        train_list.append(df_sliced.iloc[:n_rows // 2, :])
        test_list.append(df_sliced.iloc[n_rows // 2 :, :])

    train_df = pd.concat(train_list).reset_index(drop=True)
    test_df = pd.concat(test_list).reset_index(drop=True)

    X_train = train_df.iloc[:, :-1].to_numpy()
    y_train = train_df.iloc[:, -1].to_numpy()

    X_test = test_df.iloc[:, :-1].to_numpy()
    y_test = test_df.iloc[:, -1].to_numpy()

    return X_train, X_test, y_train, y_test

In [4]:
X_train, X_test, y_train, y_test = split(df, 'species')

In [5]:
BC = Bayes_Classifier(X_train, y_train, 'cccc', 'u')

In [6]:
PcX = BC.evaluate(X_test)

In [7]:
classes_predicted = np.argmax(PcX, axis=1)
(classes_predicted == y_test).sum() / y_test.size

0.92

In [8]:
(classes_predicted == y_test).sum() / y_test.size

0.92

In [9]:
Naive_Bayes_classifier(X, y, 'c', 'o')

NameError: name 'X' is not defined